In [ ]:
from racepi_database_handler import *
from racepi_can_decoder import *
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from collections import defaultdict
from ipywidgets import interact
import numpy as np
import math

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()  

In [49]:
def calc_gear_difference(trimmed_data):
    CLIP_SPEED = 26.3754 # 59 mph
    added_time = 0.0
    for p in trimmed_data:
        if p[2] > CLIP_SPEED:
            speed_ratio = CLIP_SPEED/p[2] 
            new_time = p[1] / speed_ratio
            added_time += new_time - p[1]
            p[5] = CLIP_SPEED
        p[6] = added_time
    return added_time

In [14]:
def flatten_data(data):
    can_data = [x for x in data.get('can') if x.arbitration_id == 0x80]

    tps = [[x.timestamp, focus_rs_tps_converter.convert_frame(CanFrame("999", x.msg))] for x in can_data]
    gps = data.get('gps')

    def current_tps_value(timestamp, t, ti):
        try:
            while t[0] < timestamp:
                t = ti.__next__()
        except StopIteration:
            pass
        return t
    
    flat_data = []
    t = tps[0]
    ti = iter(tps)
    for i, g in enumerate(gps):
        delta_speed=0.0
        time_delta=0.0
        if i>0:
            delta_speed = g.speed-gps[i-1].speed
            time_delta = g.timestamp-gps[i-1].timestamp
        t = current_tps_value(g.timestamp, t, ti)
        p = [g.timestamp, time_delta, g.speed, delta_speed, t[1], g.speed, 0.0]
        flat_data.append(p)
    return flat_data

def clip_flat_data(flat_data):
    TPS_MAX_THRESHOLD=65.0

    ###############################################
    # find start and end times of data and trim off

    # mark start by first time at significant speed
    start = 0
    while start < len(flat_data) and flat_data[start][2] < 10:
        start += 1

    # mark the end of the run as the last MAX TPS
    end = len(flat_data)-1
    while end>start and flat_data[end][4] < TPS_MAX_THRESHOLD:
        end -= 1;

    trimmed_data = flat_data[start:end]
    return trimmed_data

In [36]:
def get_session_data(db, session_id):
    data = defaultdict(list)
    data['gps'] = db.query(GPSData).filter(GPSData.session_id == session_id).order_by(GPSData.timestamp).all()
    data['can'] = db.query(CANData).filter(CANData.session_id == session_id).order_by(CANData.timestamp).all()
    data['imu'] = db.query(IMUData).filter(IMUData.session_id == session_id).order_by(IMUData.timestamp).all()
    return data

def load_dbfile(filename):
    engine = create_engine('sqlite:///' + filename)
    Base.metadata.bind = engine
    sm = sessionmaker()
    sm.bind = engine
    s = sm()
    return s

s = load_dbfile("/home/donour/test.db")
si = s.query(SessionInfo).filter(SessionInfo.max_speed >20).filter(SessionInfo.num_data_samples > 7000).all()


In [56]:
 
def plot(td, name):
    p = figure(title=str(name), plot_height=300, plot_width=900)
    x = [x[0]-td[0][0] for x in td]
    r1 = p.line(x, [x[2]*2.23694 for x in td], color="#2222aa", line_width=1, legend="Speed")
    r1 = p.line(x, [x[5]*2.23694 for x in td], color="#aa2222", line_width=1, legend="New Speed")
    r1 = p.line(x, [x[6]*100 for x in td], color="#aaaa22", line_width=1, legend="Time Lost (1/100)")
    r1 = p.line(x, [x[4] for x in td], color="#22aa22", line_width=1, legend="TPS")
    show(p, notebook_handle=True)

def process_session(session):    
    session_date = datetime.fromtimestamp(session.start_time_utc)
    data = get_session_data(s, session.session_id)
    fd = flatten_data(data)
    td = clip_flat_data(fd)
    v = calc_gear_difference(td)
    #plot(td, session_date)
    if v > 0.01:
        print(v, session_date, session.session_id)

for session in si[-11:-8]:
    process_session(session)

for session in si:
    #if session.session_id == "d9367540-9255-11e7-8aae-b827eb8021e9":
    process_session(session)



0.17543803161896718 2017-09-06 09:57:26.880405 56dffd80-930b-11e7-b17a-b827eb8021e9
0.09950102612987402 2017-09-06 10:04:42.813655 5abb0d54-930c-11e7-b17a-b827eb8021e9
0.22145490291763562 2017-09-06 10:32:31.721642 3d7fbd4e-9310-11e7-b64c-b827eb8021e9
0.04107057901176771 2017-07-03 09:07:17.770438 906e2854-5ff0-11e7-ac41-b827eb8021e9
0.015589244992468165 2017-07-03 09:30:13.535832 c47f0610-5ff3-11e7-ac41-b827eb8021e9
0.03172822701032213 2017-08-06 10:21:02.943766 81e11c88-7ab2-11e7-a812-b827eb8021e9
0.05619473683756072 2017-08-06 11:50:43.961730 0775d3b4-7abf-11e7-864d-b827eb8021e9
0.06273461805023839 2017-08-06 12:10:14.770670 c154d4e0-7ac1-11e7-864d-b827eb8021e9
0.06385867581589182 2017-08-06 12:31:38.638697 be968e80-7ac4-11e7-bc3b-b827eb8021e9
0.03904649030911239 2017-08-06 12:50:10.430994 5540b94e-7ac7-11e7-bc3b-b827eb8021e9
0.046453984678607424 2017-08-06 13:01:16.333618 e21be25c-7ac8-11e7-bc3b-b827eb8021e9
0.057081519305894585 2017-08-06 13:17:56.627831 36546b3a-7acb-11e7-bc3b-b8

In [6]:
# for session in si:
#     session_date = datetime.fromtimestamp(session.start_time_utc)
#     data = get_session_data(s, session.session_id)
#     fd = flatten_data(data)    
#     v = calc_gear_difference(fd)
#     if v > 0.01:
#         print(v, session_date, session.session_id)
#     #print(session_date, session.session_id, session.duration)